In [111]:
import re
# import numpy as np
from functools import reduce
from operator import mul
from bisect import insort
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
"""

### Part 1

Time to take some notes here...

(Removed for AoC rules but it's basically a transcript of the instructions)

In [110]:
class ChronospatialComputer:
    def __init__(self,txt):
        registers_match = re.findall(r'Register ([ABC]): (\d+)', txt)
        self.registers = {key: int(value) for key, value in registers_match}
        print(self.registers)
        program_match = re.search(r'Program: ([\d,]+)', txt)
        self.program = [int(x) for x in program_match.group(1).split(',')] if program_match else []
        print(self.program)

    def combo(self,op):
        if op < 4:
            return op
        elif op == 4:
            return self.registers['A']
        elif op == 5:
            return self.registers['B']
        elif op == 6:
            return self.registers['C']
        else:
            return 7

    def run_program(self,program):
        # combo operands: 0,1,2,3,A,B,C,7
        i = 0
        output = []
        while(i<len(program)):# for i in range(0,len(program),2):
            inst = program[i]
            op = program[i+1]
            jumped = False
            # print(f"A:{self.registers['A']}, B:{self.registers['B']}, C:{self.registers['C']}")
            # print(f"inst:{inst} op:{op}")
            match inst:
                case 0: # A / 2^comb_op. truncate res to int. write to A
                    res = self.registers['A'] >> self.combo(op)
                    self.registers['A'] = res
                    # print(self.combo(op))
                    # print(f"A:{self.registers['A']}")
                case 1: # bitwise xor with B and lit_op, write to B
                    res = self.registers['B'] ^ op
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 2: # comb_op % 8, write to B
                    res = self.combo(op) % 8
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 3: # if A == 0: nothing. else: pointer jumps to lit_op. does not jump 2 after
                    if self.registers['A'] != 0:
                        i = op
                        jumped = True
                        # print("pointer changed: i={i}")
                        # continue
                case 4: # bitwise XOR of B and C. ignores op, stores res in B
                    res = self.registers['B'] ^ self.registers['C']
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 5: # combo_op % 8, output separated by commas
                    res = self.combo(op) % 8
                    output.append(res)
                    # print(f"output:{output}")
                case 6: # case 0 but writes to B
                    res = self.registers['A'] >> self.combo(op)
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 7: # case 0 but writes to C
                    res = self.registers['A'] >> self.combo(op)
                    self.registers['C'] = res
                    # print(f"C:{self.registers['C']}")
            if jumped != True: i += 2
        return output

    # Part 2:
    # ref: mental-chaos (reddit) https://www.reddit.com/r/adventofcode/comments/1hg38ah/comment/m2gge90/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
    # ref: hyperneutrino (her bruteforce method helped me understand the logic) https://www.youtube.com/watch?v=BJhpteqlVPM
    def find_match(self,ans,cursor):
        sub_program = self.program[cursor:]
        # print(f"cursor={cursor},subprogram={sub_program}")
        for i in range(8):
            self.registers['A'] = ans*8+i
            self.registers['B'] = self.registers['C'] = 0
            result = self.run_program(self.program)
            # print(f"i={i} res={result} prog={sub_program}") 
            if result == sub_program: # run program always return one digit anyways            
                if cursor == 0: return ans*8+i # best case: if full solution found, return that
                # print(f"found match! recursively find: {ans*8+i}")
                ret = self.find_match(ans*8+i,cursor-1)
                if ret is not None: return ret # base case: if one branch is exhausted, return that

res = ChronospatialComputer(input)
# res.registers['C'] = 9
# res.run_instructio [2,6])
results = res.run_program(res.program)
# ''.join([str(x) + ',' for x in results])
print("part 2:",res.find_match(0,len(res.program)-1))
# res.run_program(res.program,[3*8+1,0,0])

{'A': 59590048, 'B': 0, 'C': 0}
[2, 4, 1, 5, 7, 5, 0, 3, 1, 6, 4, 3, 5, 5, 3, 0]
part 2: 105875099912602


In [32]:
# Test cases
test = ChronospatialComputer(test_input)

test.registers['C'] = 9
test.run_program([2,6])
assert test.registers['B'] == 1

test.registers['A'] = 10
ans = test.run_program([5,0,5,1,5,4])
assert ans == [0,1,2]

test.registers['A'] = 2024
ans = test.run_program([0,1,5,4,3,0])
assert ans == [4,2,5,6,7,7,7,7,3,1,0]
assert test.registers['A'] == 0

test.registers['B'] = 29
ans = test.run_program([1,7])
assert test.registers['B'] == 26
# assert ans == [4,2,5,6,7,7,7,7,3,1,0]

test.registers['B'] = 2024
test.registers['C'] = 43690
ans = test.run_program([4,0])
assert test.registers['B'] == 44354

{'A': 729, 'B': 0, 'C': 0}
[0, 1, 5, 4, 3, 0]


There were two major mistakes in my code...
1. used `^` for power instead of `**`. This was embarrassing.
2. not realizing that the output is supposed to be literally joined by commas (I assumed like past puzzles that it always takes a number...) I had to ran someone else's answer to see what the correct answer is before getting the first star. Not proud of it.

### Part 2
After a pause and some research, here's the pseudocode idea
- Every 3 bits of A represents one digit of the final output. 
- Starting from the end, passing values from `0-7` to the decoder. 
  - if it matches, recursively search the next 8 possible digits
  - else continues
- Assumes A is the only register with values that matters, B=C=0

### Takeaways
- Trying to make a dictionary when it's entirely unncessary.. is dumb. That dictionary just made me write more each time I access a register 
- When in doubt check basic operators and see if I know them correctly!
- When the incorrect results shows me "This answer isn't incorrect" without "higher/lower" hint... Consider that maybe the answer *isn't* an int?